Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



## Importações



In [1]:
#lembrar de baixar o Digraph
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [2]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [3]:
x1 = Valor(60)
print(x1)
print(x1.data)

Valor(data=60)
60


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [4]:
a = Valor(10)
b = Valor(5)

In [5]:
print(a + b) #o python não sabe ler isso sozinho

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [6]:
print(a * b) #nem isso aqui

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [7]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [8]:
a = Valor(10)
b = Valor(5)

print(a + b)
print(a * b)

print((a + b).data)
print((a * b).data)

Valor(data=15)
Valor(data=50)
15
50


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [9]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [10]:
a = Valor(10)
b = Valor(5)

print(a.data)
print(a.progenitor)

c = a + b
d = c * a

print(c)
print(a)
print(d.progenitor)

10
()
Valor(data=15)
Valor(data=10)
(Valor(data=15), Valor(data=10))


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [11]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [12]:
a = Valor(10)
b = Valor(5)

c = a + b
d = c * a

print(c)
print(c.progenitor)
print(c.operador_mae)

Valor(data=15)
(Valor(data=10), Valor(data=5))
+


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [13]:
#precisa do Digraph
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	2645770795616 [label="{ data -3.0000 }" shape=record]
	2645770795808 [label="{ data 2.0000 }" shape=record]
	2645770795376 [label="{ data 10.0000 }" shape=record]
	2645770795472 [label="{ data -6.0000 }" shape=record]
	"2645770795472*" [label="*"]
	"2645770795472*" -> 2645770795472
	2645770796000 [label="{ data 4.0000 }" shape=record]
	"2645770796000+" [label="+"]
	"2645770796000+" -> 2645770796000
	2645770795472 -> "2645770796000+"
	2645770795616 -> "2645770795472*"
	2645770795376 -> "2645770796000+"
	2645770795808 -> "2645770795472*"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [14]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [15]:
#precisa do Digraph
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

d.rotulo = "d"
e.rotulo = "e"

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	2645772023312 [label="{ d | data -6.0000 }" shape=record]
	"2645772023312*" [label="*"]
	"2645772023312*" -> 2645772023312
	2645772021920 [label="{ e | data 4.0000 }" shape=record]
	"2645772021920+" [label="+"]
	"2645772021920+" -> 2645772021920
	2645772024608 [label="{ a | data 2.0000 }" shape=record]
	2645772025712 [label="{ c | data 10.0000 }" shape=record]
	2645772023216 [label="{ b | data -3.0000 }" shape=record]
	2645772023312 -> "2645772021920+"
	2645772023216 -> "2645772023312*"
	2645772024608 -> "2645772023312*"
	2645772025712 -> "2645772021920+"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!



In [16]:
# precisa do Digraph
x1 = Valor(60, rotulo="x1")
x2 = Valor(24, rotulo="x2")
w1 = Valor(10, rotulo="w1")
w2 = Valor(5, rotulo="w2")
w3 = Valor(2, rotulo="w3")
b = Valor(7, rotulo="b")

s1 = x1 * w1
s1.rotulo = s1

s2 = x2 * w2
s2.rotulo = s2

n = s1 + s2
n.rotulo = "n"

k = n + b
k.rotulo = "k"

y = k * w3
y.rotulo = "y"

grafo = plota_grafo(y)
print(grafo)

digraph {
	graph [rankdir=LR]
	2645770795040 [label="{ Valor(data=600) | data 600.0000 }" shape=record]
	"2645770795040*" [label="*"]
	"2645770795040*" -> 2645770795040
	2645771270704 [label="{ y | data 1454.0000 }" shape=record]
	"2645771270704*" [label="*"]
	"2645771270704*" -> 2645771270704
	2645770086480 [label="{ x2 | data 24.0000 }" shape=record]
	2645770796144 [label="{ Valor(data=120) | data 120.0000 }" shape=record]
	"2645770796144*" [label="*"]
	"2645770796144*" -> 2645770796144
	2645770794128 [label="{ w2 | data 5.0000 }" shape=record]
	2645770795664 [label="{ w3 | data 2.0000 }" shape=record]
	2645770796720 [label="{ n | data 720.0000 }" shape=record]
	"2645770796720+" [label="+"]
	"2645770796720+" -> 2645770796720
	2645771271856 [label="{ k | data 727.0000 }" shape=record]
	"2645771271856+" [label="+"]
	"2645771271856+" -> 2645771271856
	2645770796864 [label="{ x1 | data 60.0000 }" shape=record]
	2645770796384 [label="{ w1 | data 10.0000 }" shape=record]
	2645770794896 [la

## Conclusão



Seguindo o assunto sobre classes em `Python`, esse experimento três, feito na mesma aula que o experimento dois, utilizamos da modelo de classes para construir o primeiro passo de uma rede neural artificial usando um grafo que trabalhamos em sala de aula, fora do JupyterLab Notebook. Podemos ver esse grafo construído na sessão "Refazendo o grafo que fizemos na aula anterior". 

Portanto, foi criada uma classe que gerasse automaticamente um grafo computacional, que representa todas as operações matemáticas relacionadas a um dado valor <i>y</i>.

Ao podemos observar ao longo do experimento, foram utilizados os mesmos modelos que no anterior, porém com o fator de operações matemáticas, as quais não conseguem ser computadas de maneira convencional, uma vez que as informações se encontram dentro da classe e a maneira que estamos acostumados a realizar os cálculos está fora dela, de forma que a máquina não consiga ler os valores chamados pelas variáveis definidas. O computador não consegue computar uma informação que ele não tem ainda, ela precisa ser dada a ele por você. Por esse motivo, foram criados os 'dunders' chamados `__add__` e `__mul__`, que simplesmente comandam que os valores de <i>a</i> e de <i>b</i> sejam somados e multiplicados, respectivamente.

Já que o intuito principal é construir grafos, precisamos definir os vértices deles, tais que contém informações que darão origem aos vértices consecutivos, chamados de progenitores. Como é necessário que essas informações sejam armazenadas, elas precisam ser colocadas dentro da classe, como visto na <i>dunder</i> `__init__`, na sessão "Registrando os progenitores", em que temos o parâmetro
`progenitor=()` referenciado na função inicial. Ele é muito importante dentro das <i>dunders</i> `__add__` e `__mul__`, para poder ir armazenando os valores calculados da rede, que estão definidos com `outro_valor`. Da mesma forma acontece com o <i><b>operador mãe</b></i>, que pode ser usado para gerar o vértice resultante do cálculo dos progenitores.

Lembre-se de rotular cada vértice, de modo a deixar o grafo plotado mais organizado e compreensível.

<b>Nota:</b> Por enquanto, qualquer grafo plotado neste Notebook não pode ser visualizado, pois meu computador não possui o software necessário para retornar a imagem dentro do JupyterLab. Caso não tenha o software em seu computador e queria ver sem precisar baixá-lo, primeiramente, certifique-se de que tenha instalado o pacote `graphviz` (pode baixá-lo usando o código presente na célula 'raw' abaixo). Depois, acesse o seguinte link https://dreampuf.github.io/GraphvizOnline/ e copie cada um dos URL's retornados pelos códigos acima e substitua o que está no script em preto pelo `digraph` que deseja ver.

## Playground

